In [1]:
pathtorepo = "C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\Tsodyks Markram\\system.jl")

UndefVarError: UndefVarError: pathtorepo not defined

In [2]:
using StaticArrays, DifferentialEquations, DynamicalSystems

E, x, y  = 0..40, 0..1,  0..1
box = E × x × y
using CairoMakie, GLMakie

In [12]:
function distance_homoclinic(p1, p2)
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    dist = sqrt( (x1-x2)^2 + (y1-y2)^2 + (z1-z2)^2 )
    return dist
end;

In [5]:
"""
arguments:
- model
- u0
- parameters
- times -- array type of [] which include time_integrate, time_transient
- integ_setting -- settings for method of integration
- t_int_ts -- time inerval for timeseries
- t_int_3d -- time inerval for 3d phase portreit
 - labels -- labels for abscissa and ordinate of timeseries
"""
function research_regime(model, u0, p, times, integ_setting;
        index_3d = [1, 2, 3], index_ts = 1,
        t_int_ts = nothing, t_int_3d = nothing,
        labels = ["time", ""], plot = true)
    
    ds = CoupledODEs(model, u0, p, diffeq = integ_setting)
    tr, trange = trajectory(ds, times[1]; Δt = integ_setting.dt, Ttr = times[2])
    

    t_int_ts = check_t_int(t_int_ts, times)
    t_int_3d = check_t_int(t_int_3d, times)

    inter_3d = interactive_3d(tr, [3, 2, 1], t_int_3d)
    ts = timeseries([trange, tr], index_ts, t_int_ts, labels)
    
    if plot == true
    GLMakie.activate!()
    display(GLMakie.Screen(), inter_3d)
    display(GLMakie.Screen(), ts)
    end
    return [ds, tr, trange, inter_3d, ts]
end
function interactive_3d(data, index, interval)
    f = Figure(resolution = (800, 500))
    axis3 = Axis3(f[1, 1],
                xlabelsize = 35, ylabelsize = 35, zlabelsize = 35,
                xticklabelsize = 20, yticklabelsize = 20, zticklabelsize = 20,
                xgridvisible = false, ygridvisible = false, zgridvisible = false)

    lines!(axis3,
        data[interval[1]:interval[2], index[1]], data[interval[1]:interval[2], index[2]],
        data[interval[1]:interval[2],index[3]], linewidth = 1.5, color = :deeppink)
    return f
end
function check_t_int(t_int, times)
    if t_int == nothing
        t_int = [1, times[1]]
    end
    return t_int
end
function timeseries(data, index, interval, labels)
    
    f = Figure(resolution = (900, 250))
    axis = Axis(f[1, 1], xlabel = labels[1], ylabel = labels[2],
            xlabelsize = 35, ylabelsize = 35,
            xticklabelsize = 28, yticklabelsize = 28,
            xgridvisible = false, ygridvisible = false)
    
    lines!(axis, data[1][interval[1]:interval[2]], data[2][interval[1]:interval[2], index],
            linewidth = 2.5, color = :deeppink)
    return f
end

timeseries (generic function with 1 method)

In [6]:
function research_regime_2_traj(model, u0_1, u0_2, p, times, integ_setting;
        index_3d = [3, 2, 1], t_int_3d = nothing,
        plot = true, onlylastpoint = false, dis_hc = false)
    
    ds_1 = CoupledODEs(model, u0_1, p, diffeq = integ_setting)
    tr_1, trange = trajectory(ds_1, times[1]; Δt = integ_setting.dt, Ttr = times[2])
    
    ds_2 = CoupledODEs(model, u0_2, p, diffeq = integ_setting)
    tr_2, _ = trajectory(ds_2, times[1]; Δt = integ_setting.dt, Ttr = times[2])
    
    fp, eigs, _ = fixedpoints(ds_1, box, jacob_TM_)
    
    if dis_hc == true
        distance_1 = zeros(length(tr_1))
        distance_2 = zeros(length(tr_2))
    
        for i in range(1, length(distance_1), step = 1)
            distance_1[i] = distance_homoclinic(tr_1[i], fp[1])
        end
    
        for i in range(1, length(distance_2), step = 1)
            distance_2[i] = distance_homoclinic(tr_2[i], fp[1])
        end
    
        dis_1 = minimum(distance_1)
        dis_2 = minimum(distance_2)
    end
    
    t_int_3d = check_t_int(t_int_3d, times)

    inter_3d = interactive_3d_2_traj(tr_1, tr_2, fp[1], index_3d, t_int_3d)
    
    if plot == true
        GLMakie.activate!()
        display(GLMakie.Screen(), inter_3d)
    end
    
    if onlylastpoint == true && dis_hc == true
        return [ds_1, ds_2, tr_1[end], tr_2[end], dis_1, dis_2, inter_3d, fp, eigs]
    elseif onlylastpoint == true
        return [ds_1, ds_2, tr_1[end], tr_2[end], dis_1, dis_2, inter_3d, fp, eigs]
        else
            return [ds_1, ds_2, tr_1, tr_2, trange, dis_1, dis_2, inter_3d, fp, eigs]
    end
    
end
function interactive_3d_2_traj(data_1, data_2, fp, index, interval)
    f = Figure(resolution = (800, 500))
    axis3 = Axis3(f[1, 1],
                xlabelsize = 35, ylabelsize = 35, zlabelsize = 35,
                xticklabelsize = 20, yticklabelsize = 20, zticklabelsize = 20,
                xgridvisible = false, ygridvisible = false, zgridvisible = false)

    lines!(axis3,
        data_1[interval[1]:interval[2], index[1]], data_1[interval[1]:interval[2], index[2]],
        data_1[interval[1]:interval[2],index[3]], linewidth = 1.0, color = :deeppink)
    
    lines!(axis3,
        data_2[interval[1]:interval[2], index[1]], data_2[interval[1]:interval[2], index[2]],
        data_2[interval[1]:interval[2],index[3]], linewidth = 1.0, color = :black)
    
    scatter!(axis3, fp[index[1]], fp[index[2]], fp[index[3]], linewidth = 1.0, color = :green)
    
    return f
end

interactive_3d_2_traj (generic function with 1 method)

In [7]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305; const U0 = 0.265;

In [8]:
time = 1000; tt = 0; tstep = 0.001
integ_set = (alg = Vern9(), adaptive = false, dt = tstep);
times = [time, tt]
time_int_3d = [1, 1000000];

In [9]:
eps_vec = [1e-14, 0.0, 1e-14];

In [10]:
I0 = -1.706
p = [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0_1 = [8.337037568242424, 0.7385031015931967, 0.4383397756224728] # output[9][1] + eps_vec
u0_2 = [19.987334710975187, 0.5898117186907356, 0.5774872394595889] # output[4][end]
println("last point  pink trajectory:$u0_1")
println("black trajectory:$u0_2")

last point  pink trajectory:[8.337037568242424, 0.7385031015931967, 0.4383397756224728]
black trajectory:[19.987334710975187, 0.5898117186907356, 0.5774872394595889]


In [13]:
output = research_regime_2_traj(TM, u0_1, u0_2, p, times, integ_set,
        t_int_3d = time_int_3d,
        plot = true, onlylastpoint = false, dis_hc = true)

10-element Vector{Any}:
  3-dimensional CoupledODEs
  3-dimensional CoupledODEs
  3-dimensional StateSpaceSet{Float64} with 1000001 points
  3-dimensional StateSpaceSet{Float64} with 1000001 points
  0.0:0.001:1000.0
 1.4609455111976626e-14
 5.5578549529902305e-5
  Figure()
  3-dimensional StateSpaceSet{Float64} with 1 points
  Vector{ComplexF64}[[-6.382803808040045 + 0.0im, 2.173275130586992 - 13.770982609518827im, 2.173275130586992 + 13.770982609518827im]]

In [14]:
data_1 = output[3]
data_2 = output[4];

In [16]:
time_int_3d = [1, 20000];

f = Figure(resolution = (900, 900))
ax = LScene(f[1, 1], show_axis = true)


scale!(ax.scene, 50, 50, 1)

lines!(data_2[time_int_3d[1]:time_int_3d[2], 3], data_2[time_int_3d[1]:time_int_3d[2], 2],
        data_2[time_int_3d[1]:time_int_3d[2], 1], linewidth = 1.0, color = :black)

display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [20]:
time_int_3d = [1, 25000];
f = Figure(resolution = (900, 900))
ax = LScene(f[1, 1], show_axis = true)
    

scale!(ax.scene, 50, 50, 1)
lines!(data_1[time_int_3d[1]:time_int_3d[2], 3], data_1[time_int_3d[1]:time_int_3d[2], 2],
        data_1[time_int_3d[1]:time_int_3d[2], 1], linewidth = 1.0, color = :deeppink)

display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [49]:
prev_1 = u0_1
prev_2 = u0_2
println("I0: $I0")
println("previous last point  pink trajectory:$prev_1")
println("previous black trajectory:$prev_2")

I0: -1.7067
previous last point  pink trajectory:[7.516774713387567, 0.746485575093425, 0.43722593278369165]
previous black trajectory:[8.30318964766908, 0.7364665354623648, 0.4379098873884975]


In [46]:
pink = lyapunovspectrum(output[1], time; Ttr = tt)
black = lyapunovspectrum(output[2], time; Ttr = tt)
println("LS  pink trajectory:$pink")
println("LS  black trajectory:$black")

LS  pink trajectory:[0.06090950633873901, 0.001442625154265159, -4.661541725120631]
LS  black trajectory:[0.0725448374823018, 0.0018920141667437512, -4.669948783685054]
